In [4]:
# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Database libraries
import psycopg2
from sqlalchemy import create_engine
import json

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [5]:
# Alternative: Direct psycopg2 connection
import psycopg2
import pandas as pd

def get_connection():
    return psycopg2.connect(
        host='localhost',
        port=5432,
        database='city_marketing',
        user='postgres',
        password='1234'
    )

try:
    # Test connection
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT 1")
    print("Direct PostgreSQL connection successful!")
    conn.close()
except Exception as e:
    print(f"Direct connection failed: {e}")
    print("Let's check container status...")

Direct PostgreSQL connection successful!


In [6]:
# Load vehicule counter data - simple direct approach
conn = get_connection()
vehicule_data = pd.read_sql("SELECT * FROM vehicle_counters", conn)
conn.close()

print(f"✅ Loaded {len(vehicule_data)} vehicule counter records")

# Quick data overview
print(f"Dataset shape: {vehicule_data.shape}")
print(f"Columns: {list(vehicule_data.columns)}")
print("\nFirst 3 rows:")
print(vehicule_data.head(3))

✅ Loaded 10000 vehicule counter records
Dataset shape: (10000, 10)
Columns: ['trajectory_id', 'site_id', 'site_label', 'mode', 'count', 'direction', 'lane', 'timestamp', 'longitude', 'latitude']

First 3 rows:
  trajectory_id site_id               site_label                 mode  count  \
0  10004_5 -> 3   10004  CF0256_88 rue de Rivoli  Autobus et autocars      4   
1  10004_1 -> 1   10004  CF0256_88 rue de Rivoli         Trottinettes      4   
2  10004_4 -> 2   10004  CF0256_88 rue de Rivoli         Trottinettes      5   

  direction                          lane                 timestamp longitude  \
0       E-O  Voie de circulation générale 2025-04-09 22:00:00+00:00  2.349109   
1       O-E                Piste cyclable 2025-04-10 00:00:00+00:00  2.349109   
2       E-O                   Coronapiste 2025-04-10 01:00:00+00:00  2.349109   

    latitude  
0  48.858273  
1  48.858273  
2  48.858273  


In [7]:
# Basic data info
print("DATASET OVERVIEW")
print("=" * 50)
print(f"Shape: {vehicule_data.shape}")
print(f"Memory usage: {vehicule_data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\nColumn Information:")
print(vehicule_data.info())

DATASET OVERVIEW
Shape: (10000, 10)
Memory usage: 4.93 MB

Column Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   trajectory_id  10000 non-null  object             
 1   site_id        10000 non-null  object             
 2   site_label     10000 non-null  object             
 3   mode           10000 non-null  object             
 4   count          10000 non-null  int64              
 5   direction      10000 non-null  object             
 6   lane           10000 non-null  object             
 7   timestamp      10000 non-null  datetime64[ns, UTC]
 8   longitude      10000 non-null  object             
 9   latitude       10000 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(8)
memory usage: 781.4+ KB
None


In [8]:
# Display first few rows
print("SAMPLE DATA")
print("=" * 50)
print(vehicule_data.head())

print("\nCOLUMN NAMES:")
print(list(vehicule_data.columns))

SAMPLE DATA
  trajectory_id site_id               site_label                     mode  \
0  10004_5 -> 3   10004  CF0256_88 rue de Rivoli      Autobus et autocars   
1  10004_1 -> 1   10004  CF0256_88 rue de Rivoli             Trottinettes   
2  10004_4 -> 2   10004  CF0256_88 rue de Rivoli             Trottinettes   
3  10004_5 -> 3   10004  CF0256_88 rue de Rivoli  Véhicules lourds > 3,5t   
4  10004_4 -> 2   10004  CF0256_88 rue de Rivoli                    Vélos   

   count direction                          lane                 timestamp  \
0      4       E-O  Voie de circulation générale 2025-04-09 22:00:00+00:00   
1      4       O-E                Piste cyclable 2025-04-10 00:00:00+00:00   
2      5       E-O                   Coronapiste 2025-04-10 01:00:00+00:00   
3      8       E-O  Voie de circulation générale 2025-04-10 02:00:00+00:00   
4     51       E-O                   Coronapiste 2025-04-10 04:00:00+00:00   

  longitude   latitude  
0  2.349109  48.858273  
1  2.3

In [9]:
# Check for missing values
print("MISSING VALUES ANALYSIS")
print("=" * 50)
missing_analysis = pd.DataFrame({
    'Missing_Count': vehicule_data.isnull().sum(),
    'Missing_Percentage': (vehicule_data.isnull().sum() / len(vehicule_data)) * 100
}).sort_values('Missing_Percentage', ascending=False)

missing_with_values = missing_analysis[missing_analysis['Missing_Count'] > 0]
if not missing_with_values.empty:
    print(missing_with_values)
else:
    print("No missing values found!")

MISSING VALUES ANALYSIS
No missing values found!
